In [19]:
import pandas as pd
import urllib.request, json
import plotly_express as px

In [3]:
with urllib.request.urlopen('https://api.mfapi.in/mf') as url:
    data = json.load(url)
df_mfs = pd.DataFrame(data)

In [10]:
df_mfs[df_mfs.schemeName.str.startswith('Parag')]

,schemeCode,schemeName
20226,122639,Parag Parikh Long Term Equity Fund - Direct Pl...
20227,122639,Parag Parikh Flexi Cap Fund - Direct Plan - Gr...
20228,122640,Parag Parikh Long Term Equity Fund - Regular P...
20229,122640,Parag Parikh Flexi Cap Fund - Regular Plan - G...
35210,143260,Parag Parikh Liquid Fund- Regular Plan- Growth
35211,143261,Parag Parikh Liquid Fund- Regular Plan- Monthl...
35212,143261,Parag Parikh Liquid Fund- Regular Plan- Monthl...
35213,143262,Parag Parikh Liquid Fund- Direct Plan- Monthly...
35214,143262,Parag Parikh Liquid Fund- Direct Plan- Monthly...
35215,143263,Parag Parikh Liquid Fund- Direct Plan- Daily D...


In [12]:
scheme_code = '122639'
mf_url = 'https://api.mfapi.in/mf/' + scheme_code
with urllib.request.urlopen(mf_url) as url:
    data = json.load(url)

In [23]:
df_navs = pd.DataFrame(data['data'])
df_navs['date'] = pd.to_datetime(df_navs.date, format='%d-%m-%Y')
df_navs = df_navs.sort_values(['date'])

In [24]:
px.line(df_navs, x = 'date', y='nav', log_y=True)